# Word2Vec с нуля (используя numpy)

В этом задании мы реализуем word2vec с нуля используя numpy.

На самом деле, word2vec -- довольно таки красивое название для алгоритма, но, по факту, любое численное представление слов можно считать вектором.

Например  `OneHotEncoding` -- это вполне себе вектор слова. Но минус такого представления в том, что мы никак не учитываем взаимосвязь слов в тексте, также каждый из векторов может очень сильно раздуться и занять много памяти.

In [2]:
from typing import List

import numpy as np

from word2vec.data import TextDataset, Vocab

In [9]:
data_path = 'word2vec/news.txt'

In [11]:
!head -n2 news.txt

head: news.txt: No such file or directory


In [13]:
ls

Icon?                   README.md               tree_based/
Introduction/           requirements.txt        word2vec/
LICENSE                 titanic_classification/


__Задание 1__

In [5]:
def preprocessor(line: str) -> List[str]:
    ### Ваш код начинается тут ###
    return line.split()
    ### Ваш код заканчивается тут ###

In [6]:
assert len(preprocessor("Все нормально")) == 2
assert isinstance(preprocessor("с типами все ок"), list)

In [7]:
vocab = Vocab()
dataset = TextDataset(data_path, preprocessor)

In [10]:
vocab.count(dataset)
vocab.build_vocab(dataset)

FileNotFoundError: [Errno 2] No such file or directory: 'news.txt'

In [8]:
dataset.vocabulary = vocab

In [9]:
len(vocab.counts)

1000

__Задание 2__

In [15]:
def generate_training_data(dataset: TextDataset, vocab: Vocab, window_size: int):
    X, Y = [], []

    for tokens in dataset:
        for i in range(len(tokens)):
    ### Ваш код начинается тут ###
#         indices = None # Тут должны быть количество window_size индексов слов слева от i и справа от i
#         for j in nbr_inds:
#             # Тут надо записать индексы из vocab.word2id в x
#             X.some_function 
#             Y.some_function 
    ### Ваш код заканчивается тут ###
            indices = list(range(max(0, i - window_size), i)) + \
                       list(range(i + 1, min(len(tokens), i + window_size + 1))) # Тут должны быть количество window_size индексов слов слева от i и справа от i
            for j in indices:
                # Тут надо записать индексы из vocab.word2id в x
                X.append(vocab.word2id[tokens[i]])
                Y.append(vocab.word2id[tokens[j]])
            
    X = np.array(X)
    X = np.expand_dims(X, axis=0)
    Y = np.array(Y)
    Y = np.expand_dims(Y, axis=0)

    return X, Y

In [16]:
X, Y = generate_training_data(dataset, vocab, 3)

In [17]:
X.shape, Y.shape

((1, 11446140), (1, 11446140))

Ожидается: 
``((1, 11446140), (1, 11446140))```

In [19]:
X[0][:3], Y[0][:3]

(array([0, 0, 0]), array([1, 2, 3]))

Ожидается: ```(array([0, 0, 0]), array([1, 2, 3]))```

In [20]:
vocab_size = len(vocab)
m = Y.shape[1]
# turn Y into one hot encoding
Y_one_hot = np.zeros((vocab_size, m))
Y_one_hot[Y.flatten(), np.arange(m)] = 1

In [21]:
Y_one_hot.shape

(1000, 11446140)

In [22]:
Y_one_hot[:3]

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

#### Задание 3

In [23]:
from optimization import Optimizer, CrossEntropy

In [24]:
class Optimizer:

    def __init__(self, model, lr=0.0001, decrease=0.98):
        self.decrease = decrease
        self.model = model
        self.lr = lr

    def optimize(self, ground_truth, pred):
        dw, dvec = self.model.grad(ground_truth, pred)
        self.model.backward({
            "embedding": dvec.T * self.lr,
            "linear": dw * self.lr
        })

    def decrease_lr(self, prod=None):
        if prod is not None:
            self.lr *= prod
        else:
            self.lr *= self.decrease

In [37]:
class Softmax:

    def forward(self, input):
        return np.exp(input) / (np.sum(np.exp(input), axis=0, keepdims=True) + 0.001)

    def grad(self, ground_truth, pred):

        pred[ground_truth.flatten(), np.arange(ground_truth.shape[1])] -= 1
        
        return pred

In [38]:
class Linear:

    def __init__(self, input_size: int, output_size: int):
        self.output_size = output_size
        self.input_size = input_size

        # В статье не используется bias
        self.W = np.random.randn(output_size, input_size) * 0.01
        self.cache = {"input": None}

    def forward(self, input):

        self.cache['input'] = input
        return self.W @ input

    def grad(self, cost):
        m = self.cache['input'].shape[1]
        dw = (1 / m) * cost @ self.cache['input'].T
        dinp = self.W.T @ cost

        assert (self.W.shape == dw.shape)
        assert (self.cache['input'].shape == dinp.shape)

        return dw, dinp

    def backward(self, step):
        self.W -= step

In [39]:
class Embedding:

    def __init__(self, vocab_size: int, emb_size):
        self.embs = np.random.randn(vocab_size, emb_size) * 0.01


    def ind2vecs(self, input):
        m = input.shape[1]
        word_vec = self.embs[input.flatten(), :].T

        assert (word_vec.shape == (self.embs.shape[1], m))

        return word_vec

    def backward(self, step, inds):
        self.embs[inds, :] -= step

In [40]:
class Word2Vec:

    def __init__(self, vocab_size: int, emb_size: int):
        self.emb_size = emb_size
        self.vocab_size = vocab_size

        self.embedding = Embedding(vocab_size, emb_size)
        self.linear = Linear(emb_size, vocab_size)
        self.softmax = Softmax()

        self.cache = {'input': None}

    def forward(self, input):
        self.cache['input'] = input

        vec = self.embedding.ind2vecs(input)
        out = self.linear.forward(vec)
        probs = self.softmax.forward(out)

        return probs

    def grad(self, ground_truth, pred):
        dz = self.softmax.grad(ground_truth, pred)
        dw, dvec = self.linear.grad(dz)
        return  dw, dvec

    def backward(self, step):
        self.embedding.backward(step['embedding'], inds=self.cache['input'].flatten())
        self.linear.backward(step['linear'])

In [57]:
ce_loss = CrossEntropy()
model = Word2Vec(vocab_size, 100)
optimizer = Optimizer(model=model, lr=0.01)

In [58]:
from tqdm import tqdm

In [59]:
def train(X, Y, model, optimizer, loss, epochs, batch_size=256,
          print_cost=True):
    costs = []
    m = X.shape[1]

    for epoch in range(epochs):
        epoch_cost = 0
        batch_inds = list(range(0, m, batch_size))
        np.random.shuffle(batch_inds)
        for i in tqdm(batch_inds):
            X_batch = X[:, i:i + batch_size]
            Y_batch = Y[:, i:i + batch_size]

            softmax_out = model.forward(X_batch)
            optimizer.optimize(Y_batch, softmax_out)
            cost = loss.forward(softmax_out, Y_batch)
            epoch_cost += np.squeeze(cost)

        costs.append(epoch_cost)
        if print_cost and epoch % (epochs // 500) == 0:
            print("Cost after epoch {}: {}".format(epoch, epoch_cost))
        if epoch % (epochs // 100) == 0:
            optimizer.decrease_lr()

    return costs


In [67]:
train(X[:, :30], Y[:, :30], model, optimizer, ce_loss, 1000, batch_size=500)




  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  



100%|██████████| 1/1 [00:00<00:00, 229.72it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 263.15it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 217.91it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 128.50it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 207.00it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 156.61it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 138.87it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 126.12it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 143.76it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 247.8

Cost after epoch 0: nan
Cost after epoch 2: nan
Cost after epoch 4: nan
Cost after epoch 6: nan
Cost after epoch 8: nan
Cost after epoch 10: nan
Cost after epoch 12: nan
Cost after epoch 14: nan
Cost after epoch 16: nan


100%|██████████| 1/1 [00:00<00:00, 227.12it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 143.37it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 262.05it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 193.43it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 278.08it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 192.94it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 229.05it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 224.28it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 392.14it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 241.84it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 214.64it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 

Cost after epoch 18: nan
Cost after epoch 20: nan
Cost after epoch 22: nan
Cost after epoch 24: nan
Cost after epoch 26: nan
Cost after epoch 28: nan
Cost after epoch 30: nan
Cost after epoch 32: nan
Cost after epoch 34: nan
Cost after epoch 36: nan
Cost after epoch 38: nan





100%|██████████| 1/1 [00:00<00:00, 217.82it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 303.03it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 143.43it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 253.23it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 214.30it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 426.08it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 281.86it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 288.43it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 267.66it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 322.99it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 207.38it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1

...




100%|██████████| 1/1 [00:00<00:00, 280.03it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 221.48it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 209.59it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 233.15it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 240.84it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 237.37it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 231.54it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 235.13it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 265.83it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 257.92it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 243.08it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1

Cost after epoch 944: nan
Cost after epoch 946: nan
Cost after epoch 948: nan
Cost after epoch 950: nan
Cost after epoch 952: nan
Cost after epoch 954: nan
Cost after epoch 956: nan
Cost after epoch 958: nan
Cost after epoch 960: nan
Cost after epoch 962: nan
Cost after epoch 964: nan
Cost after epoch 966: nan
Cost after epoch 968: nan
Cost after epoch 970: nan
Cost after epoch 972: nan





100%|██████████| 1/1 [00:00<00:00, 247.57it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 310.37it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 254.00it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 149.74it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 187.76it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 286.83it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 143.14it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 192.80it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 186.70it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 254.91it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 354.25it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1

Cost after epoch 974: nan
Cost after epoch 976: nan
Cost after epoch 978: nan
Cost after epoch 980: nan
Cost after epoch 982: nan
Cost after epoch 984: nan
Cost after epoch 986: nan
Cost after epoch 988: nan
Cost after epoch 990: nan
Cost after epoch 992: nan
Cost after epoch 994: nan
Cost after epoch 996: nan
Cost after epoch 998: nan


[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan